In [1]:
import pandas as pd
import numpy as np

In [2]:
berlin_district = {"Wedding" : "mc010",
                  "Schöneberg" : "mc018",
                  "Marienfelde" : "mc027",
                  "Grunewald" : "mc032",
                  "Neukölln" : "mc042",
                  "Buch" : "mc077",
                  "Friedrichshagen" : "mc085",
                  "Messwagen Leipziger Str." : "mw088",
                  "Hardenbergplatz" : "mc115",
                  "Schildhornstraße"  :"mc117",
                  "Mariendorfer Damm" : "mc124",
                  "Silbersteinstraße" : "mc143",
                  "Frohnau" : "mc145",
                  "Mitte" : "mc171",
                  "Frankfurter Allee" : "mc174",
                  "Karl-Marx-Straße" : "mc220",
                  "Karlshorst" : "mc282"}

berlin_period = {"Dayly" : "24h",
                "Hourly" : "1h",
                "Monthly" : "1m"}



In [3]:
# load multiple csv of website, for all districts of berlin
period  = berlin_period["Dayly"]
start = "01.01.2000"
end = "30.10.2019"
new_df=pd.DataFrame()
for key, value in berlin_district.items():
    csv_downloadlink = "https://luftdaten.berlin.de/station/{0}.csv?group=pollution&period={1}&timespan=custom&start%5Bdate%5D={2}&start%5Bhour%5D=00&end%5Bdate%5D={3}&end%5Bhour%5D=00".format(value, period, start, end)
    district = pd.read_csv(csv_downloadlink, sep=";", header=[1,2])
    district = district.drop([district.index[0]])
    #district = district.assign(Ortschaft = key).
    district.columns = [f'{i}{j}' for i, j in district.columns]
    district = district.rename(columns={"Feinstaub (PM10)µg/m³" : "PM10",
                                    "Kohlenmonoxidmg/m³" : "CO",
                                    "Stickoxideµg/m³": "NOx",
                                    "Stickstoffdioxidµg/m³" : "NO_2", 
                                    "Ozonµg/m³" : "O_3",
                                    "Stickstoffmonoxidµg/m³" : "NO",
                                    "Schwefeldioxidµg/m³" : "SO_2", 
                                    "Ortschaft" : "city",
                                    "MesskomponenteEinheit" : "date"})
    if "CO" not in list(district.columns):
        district=district.assign(CO="nan")
    if "SO_2" not in list(district.columns):
        district=district.assign(SO_2="nan")
    if "PM10" not in list(district.columns):
        district=district.assign(PM10="nan")
    if "O_3" not in list(district.columns):
        district=district.assign(O_3="nan")
    district=district[["date","CO","NO_2","NOx", "O_3", "PM10", "SO_2"]]
    new_df=new_df.append(district,ignore_index=True)
    #district.to_csv(path_or_buf="././Berlin_Data/"+value +".csv")

In [4]:
new_df

,date,CO,NO_2,NOx,O_3,PM10,SO_2
0,29.10.2009,nan,31,45,10,18,nan
1,30.10.2009,nan,35,60,11,17,nan
2,31.10.2009,nan,30,40,17,28,nan
3,01.11.2009,nan,25,35,13,47,nan
4,02.11.2009,nan,36,62,7,34,nan
...,...,...,...,...,...,...,...
59154,26.10.2019,nan,11,13,nan,nan,NaN
59155,27.10.2019,nan,14,16,nan,nan,NaN
59156,28.10.2019,nan,27,36,nan,nan,NaN
59157,29.10.2019,nan,31,55,nan,nan,NaN


In [5]:
new_df["date"] = pd.to_datetime(new_df["date"], infer_datetime_format=True)

In [6]:
new_df

,date,CO,NO_2,NOx,O_3,PM10,SO_2
0,2009-10-29,nan,31,45,10,18,nan
1,2009-10-30,nan,35,60,11,17,nan
2,2009-10-31,nan,30,40,17,28,nan
3,2009-11-01,nan,25,35,13,47,nan
4,2009-11-02,nan,36,62,7,34,nan
...,...,...,...,...,...,...,...
59154,2019-10-26,nan,11,13,nan,nan,NaN
59155,2019-10-27,nan,14,16,nan,nan,NaN
59156,2019-10-28,nan,27,36,nan,nan,NaN
59157,2019-10-29,nan,31,55,nan,nan,NaN


In [7]:
new_df=new_df.astype({"date":"datetime64","CO":"float","NO_2":"float","NOx":"float", "O_3":"float", "PM10":"float", "SO_2":"float"})

In [8]:
new_df.dtypes

date    datetime64[ns]
CO             float64
NO_2           float64
NOx            float64
O_3            float64
PM10           float64
SO_2           float64
dtype: object

In [9]:
new_df = new_df.groupby("date").mean().reset_index()
new_df["city"] = "berlin"
new_df

,date,CO,NO_2,NOx,O_3,PM10,SO_2,city
0,2009-10-29,0.500000,33.812500,71.687500,16.142857,19.583333,1.5,berlin
1,2009-10-30,0.700000,39.312500,94.437500,15.571429,21.833333,1.5,berlin
2,2009-10-31,0.650000,33.312500,70.875000,18.571429,32.750000,4.0,berlin
3,2009-11-01,0.600000,25.000000,48.625000,18.285714,50.333333,6.0,berlin
4,2009-11-02,0.850000,39.312500,95.812500,13.000000,35.916667,4.0,berlin
...,...,...,...,...,...,...,...,...
3649,2019-10-26,0.300000,20.117647,35.058824,38.666667,13.833333,1.0,berlin
3650,2019-10-27,0.233333,17.000000,26.294118,46.666667,11.750000,1.0,berlin
3651,2019-10-28,0.333333,28.176471,56.647059,34.000000,12.083333,2.0,berlin
3652,2019-10-29,0.266667,30.411765,52.647059,24.444444,14.166667,0.0,berlin


In [10]:
new_df["PM25"] = np.nan
poll_ber = new_df[["date","CO","NO_2","NOx", "O_3", "PM10", "PM25", "SO_2","city"]]

In [11]:
poll_ber

,date,CO,NO_2,NOx,O_3,PM10,PM25,SO_2,city
0,2009-10-29,0.500000,33.812500,71.687500,16.142857,19.583333,NaN,1.5,berlin
1,2009-10-30,0.700000,39.312500,94.437500,15.571429,21.833333,NaN,1.5,berlin
2,2009-10-31,0.650000,33.312500,70.875000,18.571429,32.750000,NaN,4.0,berlin
3,2009-11-01,0.600000,25.000000,48.625000,18.285714,50.333333,NaN,6.0,berlin
4,2009-11-02,0.850000,39.312500,95.812500,13.000000,35.916667,NaN,4.0,berlin
...,...,...,...,...,...,...,...,...,...
3649,2019-10-26,0.300000,20.117647,35.058824,38.666667,13.833333,NaN,1.0,berlin
3650,2019-10-27,0.233333,17.000000,26.294118,46.666667,11.750000,NaN,1.0,berlin
3651,2019-10-28,0.333333,28.176471,56.647059,34.000000,12.083333,NaN,2.0,berlin
3652,2019-10-29,0.266667,30.411765,52.647059,24.444444,14.166667,NaN,0.0,berlin


In [12]:
poll_ber.to_pickle('./pickles/berlin_polluion.pkl')